In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import types
from ..p_utils import get_layer_metric_array, reshape_element


In [2]:
import json
# Load mean and standard deviaiton for data normalization and store in list
file_path = r'/home/ansonhon/mase_project/machop/proxy_mean_stddev.json'
with open(file_path, 'r') as file:
    proxy_mean_stddev = json.load(file)

In [3]:
proxy_mean_stddev

{'epe_nas': {'mean': 333.2484577323006, 'stddev': 100.5959189176116},
 'fisher': {'mean': 2.430095713934861, 'stddev': 3.7407969600812327},
 'grad_norm': {'mean': 60.52966159582138, 'stddev': 57.106270064853845},
 'grasp': {'mean': 91.55332415550947, 'stddev': 167.16025181714858},
 'jacov': {'mean': -149.45234642972576, 'stddev': 294.9420185798349},
 'l2_norm': {'mean': 167.9578857421875, 'stddev': 66.29851714062738},
 'nwot': {'mean': 375.1082852721538, 'stddev': 37.76466803582916},
 'plain': {'mean': 0.03036989737302065, 'stddev': 0.1357033897978934},
 'snip': {'mean': 107.75668156147003, 'stddev': 102.38174169845342},
 'synflow': {'mean': 52.40235961317503, 'stddev': 21.545154099268107},
 'zen': {'mean': 65.06115800142288, 'stddev': 43.06522364304144},
 'flops': {'mean': 44.823232000000004, 'stddev': 23.95783355013554},
 'params': {'mean': 0.31158600000000003, 'stddev': 0.16377558792445226}}

In [ ]:
def fisher_forward_conv2d(self, x):
    x = F.conv2d(
        x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups
    )
    # intercept and store the activations after passing through 'hooked' identity op
    self.act = self.dummy(x)
    return self.act


def fisher_forward_linear(self, x):
    x = F.linear(x, self.weight, self.bias)
    self.act = self.dummy(x)
    return self.act


# @measure("fisher", bn=True, mode="channel")
def compute_fisher_per_weight(net, inputs, targets, loss_fn, mode, split_data=1):

    device = inputs.device

    if mode == "param":
        raise ValueError("Fisher pruning does not support parameter pruning.")

    net.train()
    all_hooks = []
    for layer in net.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            # variables/op needed for fisher computation
            layer.fisher = None
            layer.act = 0.0
            layer.dummy = nn.Identity()

            # replace forward method of conv/linear
            if isinstance(layer, nn.Conv2d):
                layer.forward = types.MethodType(fisher_forward_conv2d, layer)
            if isinstance(layer, nn.Linear):
                layer.forward = types.MethodType(fisher_forward_linear, layer)

            # function to call during backward pass (hooked on identity op at output of layer)
            def hook_factory(layer):
                def hook(module, grad_input, grad_output):
                    act = layer.act.detach()
                    grad = grad_output[0].detach()
                    if len(act.shape) > 2:
                        g_nk = torch.sum((act * grad), list(range(2, len(act.shape))))
                    else:
                        g_nk = act * grad
                    del_k = g_nk.pow(2).mean(0).mul(0.5)
                    if layer.fisher is None:
                        layer.fisher = del_k
                    else:
                        layer.fisher += del_k
                    del (
                        layer.act
                    )  # without deleting this, a nasty memory leak occurs! related: https://discuss.pytorch.org/t/memory-leak-when-using-forward-hook-and-backward-hook-simultaneously/27555

                return hook

            # register backward hook on identity fcn to compute fisher info
            layer.dummy.register_backward_hook(hook_factory(layer))

    N = inputs.shape[0]
    for sp in range(split_data):
        st = sp * N // split_data
        en = (sp + 1) * N // split_data

        net.zero_grad()
        outputs = net(inputs[st:en])
        loss = loss_fn(outputs, targets[st:en])
        loss.backward()

    # retrieve fisher info
    def fisher(layer):
        if layer.fisher is not None:
            return torch.abs(layer.fisher.detach())
        else:
            return torch.zeros(layer.weight.shape[0])  # size=ch

    grads_abs_ch = get_layer_metric_array(net, fisher, mode)

    # broadcast channel value here to all parameters in that channel
    # to be compatible with stuff downstream (which expects per-parameter metrics)
    # TODO cleanup on the selectors/apply_prune_mask side (?)
    shapes = get_layer_metric_array(net, lambda l: l.weight.shape[1:], mode)

    grads_abs = reshape_elements(grads_abs_ch, shapes, device)

    return grads_abs

In [10]:
%pwd

'/home/ansonhon/mase_project/machop/naslib/predictors/utils/pruners/measures'

In [ ]:
### Define dummy model
class NeuralModel(nn.Module):
    def __init__(self):
        super(NeuralModel, self).__init__()
        self.linear1 = nn.Linear(13, 64)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(64, 128)
        self.relu = nn.ReLU()
        self.linear3 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))  # 输出层的激活函数使用sigmoid
        return x